**Importing libraries**

In [2]:
import numpy as np 
import pandas as pd 
import multiprocessing as multi
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output

**Capturing name of the PC**

In [3]:
%%capture PC_name 
!hostname

**Setting notebook parameters**

In [4]:
#setting dots per inch for images
dpi = 90

#setting number of processors for multiprocessing
PC_name_str = PC_name.stdout[0:-2]

if PC_name_str == 'science12':
    cores = 42        
else:
    cores = multi.cpu_count()

print(PC_name_str)
print(cores)

science12
42


**Defining constants**

In [5]:
omega_m = 0.272            #matter density paratmeter from Komatsu et al. (2011) 
omega_l = 1 - omega_m      #vacuum density paratmeter assuming flat universe
H_o = 70.4                 #Hubble constant in km s^−1 Mpc^−1 from Komatsu et al. (2011) 
h = 0.704
f = omega_m**0.545         #linear velocity growth rate from Tanimura et al. (2020)


**Defining Functions**

In [6]:
def H(z):                           #hubble parameter (in km s^-1 Mpc^-1) using Eq. 4.33 in Peter's book
    return np.sqrt( H_o**2 * ( (1+z)**3 * omega_m + omega_l ) ) 


**Reading galaxies dataset**

In [14]:

z = 0.42372720
# data_address = '../input/magneticumsnap027z042-massfiltered/'
data_address = 'Data/'
df_gal = pd.read_csv(data_address + 'massive_galaxies.csv')      #massive_galaxies.csv contains galaxies with mass greater than 1.8 ×10^{11} h^{−1} M_sun as done by Tanimura et al. (2020)
df_gal.describe()


,x[kpc/h],y[kpc/h],z[kpc/h],m[Msol/h]
count,93097.000000,93097.000000,93097.000000,9.309700e+04
mean,325576.133087,322500.028738,320613.762863,3.993206e+11
std,184863.273960,182799.587421,183249.915097,3.995129e+11
min,1.864417,10.164207,0.260974,1.800010e+11
25%,163866.950000,166201.380000,162514.560000,2.161480e+11
50%,332449.620000,329260.160000,319925.780000,2.784900e+11
75%,483949.000000,475466.120000,478698.410000,4.192930e+11
max,639996.190000,639980.940000,639993.120000,1.077460e+13


In [13]:
df_gal_temp = df_gal.copy()

df_gal_big = pd.DataFrame()

for k in range(3):
    for i in range(3):
        for j in range(0,3):

            df_gal_temp['x[kpc/h]'] = df_gal['x[kpc/h]'] + (640000 * i)
            df_gal_temp['y[kpc/h]'] = df_gal['y[kpc/h]'] + (640000 * j)
            df_gal_temp['z[kpc/h]'] = df_gal['z[kpc/h]'] + (640000 * k)

            df_gal_big = df_gal_big.append(df_gal_temp)

df_gal_big.describe()

,x[kpc/h],y[kpc/h],z[kpc/h],m[Msol/h]
count,2.513619e+06,2.513619e+06,2.513619e+06,2.513619e+06
mean,9.655761e+05,9.625000e+05,9.606138e+05,3.993206e+11
std,5.542931e+05,5.536083e+05,5.537571e+05,3.995109e+11
min,1.864417e+00,1.016421e+01,2.609741e-01,1.800010e+11
25%,4.839490e+05,4.754661e+05,4.786984e+05,2.161480e+11
50%,9.724496e+05,9.692602e+05,9.599258e+05,2.784900e+11
75%,1.443867e+06,1.446201e+06,1.442515e+06,4.192930e+11
max,1.919996e+06,1.919981e+06,1.919993e+06,1.077460e+13


**Reading clusters dataset**

In [8]:
df_clusters_orig = pd.read_csv(data_address + 'massive_clusters.csv', 
                          usecols = ['x[kpc/h]', 'y[kpc/h]', 'z[kpc/h]', 'm500c[Msol/h]', 'vx[km/s]', 'vy[km/s]', 'vz[km/s]'    ])  #massive_clusters.csv contains clusters with M_500c greater than 10^13.5 h^{-1} M_sun as done by Tanimura et al. (2020)

df_clusters_orig.describe()

,x[kpc/h],y[kpc/h],z[kpc/h],m500c[Msol/h],vx[km/s],vy[km/s],vz[km/s]
count,6080.000000,6080.000000,6080.000000,6.080000e+03,6080.000000,6080.000000,6080.000000
mean,326416.401838,322055.402286,321308.240244,6.356245e+13,2.109102,-1.647297,-4.296055
std,186021.550694,182650.899538,183665.854310,4.851698e+13,316.064539,306.226537,286.454388
min,19.163288,152.653880,87.370949,3.162340e+13,-1179.530000,-1163.520000,-1104.790000
25%,160978.575000,164793.430000,161863.925000,3.795035e+13,-200.229000,-205.139250,-190.806000
50%,338884.565000,333274.345000,322110.075000,4.801630e+13,-3.619450,1.969450,-8.055475
75%,486227.747500,473556.765000,478893.610000,6.945502e+13,206.816750,207.755750,181.532000
max,639887.560000,639933.380000,639848.380000,7.438200e+14,1197.640000,1164.960000,1201.260000


In [12]:
df_clusters_temp = df_clusters_orig.copy()

df_clusters_big = pd.DataFrame()

for k in range(3):
    for i in range(3):
        for j in range(0,3):

            df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * i)
            df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * j)
            df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * k)

            df_clusters_big = df_clusters_big.append(df_clusters_temp)

df_clusters_big.describe()

,x[kpc/h],y[kpc/h],z[kpc/h],m500c[Msol/h],vx[km/s],vy[km/s],vz[km/s]
count,1.641600e+05,1.641600e+05,1.641600e+05,1.641600e+05,164160.000000,164160.000000,164160.000000
mean,9.664164e+05,9.620554e+05,9.613082e+05,6.356245e+13,2.109102,-1.647297,-4.296055
std,5.546773e+05,5.535561e+05,5.538918e+05,4.851314e+13,316.039508,306.202286,286.431703
min,1.916329e+01,1.526539e+02,8.737095e+01,3.162340e+13,-1179.530000,-1163.520000,-1104.790000
25%,4.863366e+05,4.736062e+05,4.789278e+05,3.795035e+13,-200.229000,-205.139250,-190.806000
50%,9.788846e+05,9.732743e+05,9.621101e+05,4.801630e+13,-3.619450,1.969450,-8.055475
75%,1.440918e+06,1.444771e+06,1.441655e+06,6.945502e+13,206.816750,207.755750,181.532000
max,1.919888e+06,1.919933e+06,1.919848e+06,7.438200e+14,1197.640000,1164.960000,1201.260000


In [11]:
df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 0)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 0)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 0)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 1)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 0)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 2)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 1)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 0)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 1)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 1)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 1)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 2)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 2)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 0)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 2)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 1)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

df_clusters_temp['x[kpc/h]'] = df_clusters_orig['x[kpc/h]'] + (640000 * 2)
df_clusters_temp['y[kpc/h]'] = df_clusters_orig['y[kpc/h]'] + (640000 * 2)
df_clusters_temp['z[kpc/h]'] = df_clusters_orig['z[kpc/h]'] + (640000 * 0)

df_clusters_big = df_clusters_orig.append(df_clusters_temp)

**Removing clusters at edges of the simulation box**

In [ ]:
def edge_clus_remover(clus_cube_size):
    
    low_bound = clus_cube_size//2
    upp_bound = 640000 - (clus_cube_size//2)
    
    df_clusters = df_clusters_orig[(df_clusters_orig['x[kpc/h]'] > low_bound) & 
                                   (df_clusters_orig['x[kpc/h]'] < upp_bound) & 
                                   (df_clusters_orig['y[kpc/h]'] > low_bound) & 
                                   (df_clusters_orig['y[kpc/h]'] < upp_bound) & 
                                   (df_clusters_orig['z[kpc/h]'] > low_bound) & 
                                   (df_clusters_orig['z[kpc/h]'] < upp_bound)]
    
    return df_clusters

**Specifying prefactors for Eq. 1 of Tanimura et al. (2020)**

In [ ]:
a = 1/(1+z)
H(z)
print(H(z))

pre_fac = (f * a * H(z) / (4 * np.pi))           #in km s^−1 Mpc^−1 
pre_fac

**Calculating mean density of the simulation box for Eq. 1**

In [ ]:
def delta_gal_mean_func(cell_size):
    
    df_gal_mean = df_gal.copy()

    df_gal_mean['x[kpc/h]'] = df_gal_mean['x[kpc/h]'] / cell_size
    df_gal_mean['y[kpc/h]'] = df_gal_mean['y[kpc/h]'] / cell_size
    df_gal_mean['z[kpc/h]'] = df_gal_mean['z[kpc/h]'] / cell_size

    #making tuples, converting tuples to cell coordinates
    df_gal_mean["cell"] = list(zip(df_gal_mean['x[kpc/h]'].astype(int), df_gal_mean['y[kpc/h]'].astype(int), df_gal_mean['z[kpc/h]'].astype(int)))

    #array to store number of galaxies in the cells
    gals_in_cell = np.zeros((640000//cell_size, 640000//cell_size, 640000//cell_size))
    
    #counting number of galaxies in the cells
    for cell in df_gal_mean["cell"]:
        x, y, z = cell
        gals_in_cell[x, y, z] += 1

    delta_gal_mean = np.mean(gals_in_cell)
    
    return delta_gal_mean

**Calculating Overdensity field for Eq. 1**

In [ ]:
b = 2                                #bias for LOWZ & CMASS galaxies as taken by Tanimura et al. 2020

def overdensity_field_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, sigma_in_pix, clus_cube_size):

    #converting strings into floats
    clus_x = float(clus_x); clus_y = float(clus_y); clus_z = float(clus_z)
    
    #filtering galaxies in a cube of 240,000 h^-1 kpc around the given cluster
    df_gal_select = df_gal[(df_gal['x[kpc/h]'] > (clus_x - clus_cube_size//2)) & 
                           (df_gal['x[kpc/h]'] < (clus_x + clus_cube_size//2)) & 
                           (df_gal['y[kpc/h]'] > (clus_y - clus_cube_size//2)) & 
                           (df_gal['y[kpc/h]'] < (clus_y + clus_cube_size//2)) & 
                           (df_gal['z[kpc/h]'] > (clus_z - clus_cube_size//2)) & 
                           (df_gal['z[kpc/h]'] < (clus_z + clus_cube_size//2))]
    
    #making copy to extract coordinates of cells containing the galaxies
    df_gal_cube = df_gal_select.copy()
    
    #moving the galxies cube to lie within 0 to 240,000 h^-1 kpc 
    df_gal_cube['x[kpc/h]'] -= (clus_x - clus_cube_size//2)
    df_gal_cube['y[kpc/h]'] -= (clus_y - clus_cube_size//2)
    df_gal_cube['z[kpc/h]'] -= (clus_z - clus_cube_size//2)

    #dividing by 5000 (integer-div) so we get cell coordinates
    df_gal_cube['x[kpc/h]'] = df_gal_cube['x[kpc/h]'] / cell_size
    df_gal_cube['y[kpc/h]'] = df_gal_cube['y[kpc/h]'] / cell_size
    df_gal_cube['z[kpc/h]'] = df_gal_cube['z[kpc/h]'] / cell_size
    
    #making tuples, converting tuples to cell coordinates
    df_gal_cube["cell"] = list(zip(df_gal_cube['x[kpc/h]'].astype(int), df_gal_cube['y[kpc/h]'].astype(int), df_gal_cube['z[kpc/h]'].astype(int)))
    
    #array to store number of galaxies in the cells
    gals_in_cell = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    
    #counting number of galaxies in the cells
    for cell in df_gal_cube["cell"]:
        x, y, z = cell
        gals_in_cell[x, y, z] += 1
        
    #determining the overdensity of galaxies    
    delta_gal = (gals_in_cell/delta_gal_mean) - 1
    
    #smoothing the overdensity of galaxies
    delta_gal_smooth = gaussian_filter(delta_gal, sigma = sigma_in_pix)
        
    #obtaining matter overdensity from galaxies overdensity
    delta_matter = delta_gal_smooth / b
    
    return delta_matter


**Calculating differential, numerator & denominator for Eq. 1**

In [ ]:
def vel_terms_calc(cell_size, clus_cube_size):
    
    #calculating the differential in the Eq. 1
    dy_cubed = cell_size**3
    
    #specifing position of the clusters
    Rclus_x = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_x[:] = clus_cube_size//2
    Rclus_y = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_y[:] = clus_cube_size//2
    Rclus_z = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rclus_z[:] = clus_cube_size//2

    #generating meshgrid containing coordinates of the centers of cells
    Rcell_x = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rcell_y = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))
    Rcell_z = np.zeros((clus_cube_size//cell_size, clus_cube_size//cell_size, clus_cube_size//cell_size))

    for i, val in enumerate(range(cell_size//2, clus_cube_size, cell_size)):
        Rcell_x[i,:,:] = val
        Rcell_y[:,i,:] = val
        Rcell_z[:,:,i] = val

    #evaluating the term in the denominator of Eq. 1 of Tanimura et al. 2020
    denom = np.sqrt((Rcell_x - Rclus_x)**2 + (Rcell_y - Rclus_y)**2 + (Rcell_z - Rclus_z)**2)**(3)

    #evaluating the direction term in the numerator of Eq. 1
    numer_x = Rcell_x - Rclus_x
    numer_y = Rcell_y - Rclus_y
    numer_z = Rcell_z - Rclus_z
    
    return (dy_cubed, numer_x, numer_y, numer_z, denom)


**Calculating velocity of clusters according to Eq. 1**

In [ ]:
def clus_velocity_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, vel_terms, sigma_in_pix, 
                       clus_cube_size):
    
    delta_matter = overdensity_field_calc(clus_x, clus_y, clus_z, delta_gal_mean, cell_size, sigma_in_pix, 
                                          clus_cube_size)
    
    
    dy_cubed, numer_x, numer_y, numer_z, denom = vel_terms
       
    #estimating velocity in x direction
    integrand_x = dy_cubed * delta_matter * (numer_x/(h*1e3)) / denom #in units of Mpc    
    vx_est = pre_fac * np.sum(integrand_x)
        
    #estimating velocity in y direction
    integrand_y = dy_cubed * delta_matter * (numer_y/(h*1e3)) / denom #in units of Mpc    
    vy_est = pre_fac * np.sum(integrand_y)
    
    #estimating velocity in z direction
    integrand_z = dy_cubed * delta_matter * (numer_z/(h*1e3)) / denom #in units of Mpc    
    vz_est = pre_fac * np.sum(integrand_z)
    
    return(vx_est, vy_est, vz_est)


**Calculating sigma for smoothing**

In [ ]:
def sigma_calc(cell_size):
    
    FWHM = 2000                           #h^-1 kpc, of Gaussian kernel, taken by Tanimura et al. 2020
    FWHM_in_pix = FWHM/cell_size          #in pixel units
    sigma_in_pix = FWHM_in_pix/(2.35482)  #in pixel units
    
    return sigma_in_pix

**Plotting the scatter plots & histograms for assesment of velocity estimates **

In [ ]:
def plotting_func(df_clusters_est_err, cell_size, clus_cube_size_actual):
    
    fig = plt.figure(dpi = dpi, figsize = (10,12))

    plt.subplot(3,2,1)
    plt.scatter(df_clusters_est_err['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'], s = 8)
    plt.xlabel('V$_\mathrm{x, true}$ (km/s)')
    plt.ylabel('V$_\mathrm{x, estimated}$ (km/s)')
    plt.gca().set_xticks(range(-2000, 2001, 1000))
    plt.gca().set_yticks(range(-2000, 2001, 1000))
    plt.ylim(-2000,2000)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Vx - scatter plot')

    plt.subplot(3,2,2)
    error_x = df_clusters_est_err['vx[km/s]'] - df_clusters_est_err['vx_est[km/s]']
    plt.hist(error_x, bins = 100)
    plt.xlabel('Error in V$_{\mathrm{x}}$ (km/s)')
    plt.ylabel('Number of clusters')
    x_low, x_high = plt.xlim()
    plt.xlim(x_low, abs(x_low))

    minus_one = {-1}
    plt.text(0.57, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes)
    plt.text(0.57, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes)
    plt.text(0.6, 0.76, f'Mean: {round(np.mean(error_x), 1)} km/s', transform=plt.gca().transAxes)
    plt.text(0.6, 0.69, f'SD: {round(np.std(error_x), 1)} km/s', transform=plt.gca().transAxes)
    r_vx = np.corrcoef(df_clusters_est_err['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'])[1,0]
    plt.text(0.6, 0.62, f'Pearson\'s r: {round(r_vx, 2)}', transform=plt.gca().transAxes)

    plt.title('Vx error - histogram')




    plt.subplot(3,2,3)
    plt.scatter(df_clusters_est_err['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'], s = 8)
    plt.xlabel('V$_\mathrm{y, true}$ (km/s)')
    plt.ylabel('V$_\mathrm{y, estimated}$ (km/s)')
    plt.gca().set_xticks(range(-2000, 2001, 1000))
    plt.gca().set_yticks(range(-2000, 2001, 1000))
    plt.ylim(-2000,2000)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Vy - scatter plot')

    plt.subplot(3,2,4)
    error_y = df_clusters_est_err['vy[km/s]'] - df_clusters_est_err['vy_est[km/s]']
    plt.hist(error_y, bins = 100)
    plt.xlabel('Error in V$_{\mathrm{y}}$ (km/s)')
    plt.ylabel('Number of clusters')
    x_low, x_high = plt.xlim()
    plt.xlim(x_low, abs(x_low))

    plt.text(0.57, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes)
    plt.text(0.57, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes)
    plt.text(0.6, 0.76, f'Mean: {round(np.mean(error_y), 1)} km/s', transform=plt.gca().transAxes)
    plt.text(0.6, 0.69, f'SD: {round(np.std(error_y), 1)} km/s', transform=plt.gca().transAxes)
    r_vy = np.corrcoef(df_clusters_est_err['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'])[1,0]
    plt.text(0.6, 0.62, f'Pearson\'s r: {round(r_vy, 2)}', transform=plt.gca().transAxes)

    plt.title('Vy error - histogram')




    plt.subplot(3,2,5)
    plt.scatter(df_clusters_est_err['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'], s = 8)
    plt.xlabel('V$_\mathrm{z, true}$ (km/s)')
    plt.ylabel('V$_\mathrm{z, estimated}$ (km/s)');
    plt.gca().set_xticks(range(-2000, 2001, 1000))
    plt.gca().set_yticks(range(-2000, 2001, 1000))
    plt.ylim(-2000,2000)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.title('Vz - scatter plot')

    plt.subplot(3,2,6)
    error_z = df_clusters_est_err['vz[km/s]'] - df_clusters_est_err['vz_est[km/s]']
    plt.hist(error_z, bins = 100)
    plt.xlabel('Error in V$_{\mathrm{z}}$ (km/s)')
    plt.ylabel('Number of clusters')
    x_low, x_high = plt.xlim()
    plt.xlim(x_low, abs(x_low))

    plt.text(0.57, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes)
    plt.text(0.57, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes)
    plt.text(0.6, 0.76, f'Mean: {round(np.mean(error_z), 1)} km/s', transform=plt.gca().transAxes)
    plt.text(0.6, 0.69, f'SD: {round(np.std(error_z), 1)} km/s', transform=plt.gca().transAxes)
    r_vz = np.corrcoef(df_clusters_est_err['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'])[1,0]
    plt.text(0.6, 0.62, f'Pearson\'s r: {round(r_vz, 2)}', transform=plt.gca().transAxes)

    plt.title('Vz error - histogram')

    os.system(f'mkdir Plots/v_scatter_hist/{clus_cube_size_actual}')
    plt.subplots_adjust(top = 0.9, hspace = 0.4, wspace = 0.3)
    plt.tight_layout()
    plt.savefig(f'Plots/v_scatter_hist/{clus_cube_size_actual}/v_scatter_hist_{cell_size}.png')
    plt.close()

**Examining the effects of cluster cube & cell sizes variation on velocity estimates**

In [ ]:
%%time
cell_size = 10000                      #h^-1 kpc, size of pixel or cell

cube_cell_size_assess = pd.DataFrame(columns=['Cell Size', 'Act Cube Size', 'Cube Size Set', 'Clusters', 'Mean - Vx', 'Mean - Vy', 
                                              'Mean - Vz', 'SD - Vx', 'SD - Vy', 'SD - Vz', 'r - Vx', 'r - Vy','r - Vz'])

for clus_cube_size in [160000, 200000, 240000, 280000, 320000]:
    for cell_size in [2000, 4000, 5000, 8000, 10000, 20000]:
        
        print(cell_size, clus_cube_size)
                
        no_of_cells = clus_cube_size//cell_size
        
        clus_cube_size_actual = clus_cube_size
        
        if no_of_cells % 2 != 0:
            clus_cube_size = clus_cube_size + cell_size
        
        sigma_in_pix = sigma_calc(cell_size)

        df_clusters = edge_clus_remover(clus_cube_size)

        delta_gal_mean = delta_gal_mean_func(cell_size)

        vel_terms = vel_terms_calc(cell_size, clus_cube_size)

        clus_param = list(zip(df_clusters['x[kpc/h]'], df_clusters['y[kpc/h]'], df_clusters['z[kpc/h]'], 
                              [delta_gal_mean]*len(df_clusters), [cell_size]*len(df_clusters), 
                              [vel_terms]*len(df_clusters), [sigma_in_pix]*len(df_clusters),
                              [clus_cube_size]*len(df_clusters)))

        pool = multi.Pool(processes = cores)
        v_est = pool.starmap(clus_velocity_calc, clus_param)

        df_clusters_est_err = df_clusters.copy()

        df_clusters_est_err['vx_est[km/s]'] = [i[0] for i in v_est]
        df_clusters_est_err['vy_est[km/s]'] = [i[1] for i in v_est]
        df_clusters_est_err['vz_est[km/s]'] = [i[2] for i in v_est]

        df_clusters_est_err['vx_err[km/s]'] = df_clusters_est_err['vx[km/s]'] - df_clusters_est_err['vx_est[km/s]']
        df_clusters_est_err['vy_err[km/s]'] = df_clusters_est_err['vy[km/s]'] - df_clusters_est_err['vy_est[km/s]']
        df_clusters_est_err['vz_err[km/s]'] = df_clusters_est_err['vz[km/s]'] - df_clusters_est_err['vz_est[km/s]']
        
        plotting_func(df_clusters_est_err, cell_size, clus_cube_size_actual)
        
        clear_output(wait=False)

        cube_cell_size_assess = cube_cell_size_assess.append({'Cell Size': cell_size,
                'Act Cube Size': clus_cube_size_actual,
                'Cube Size Set': clus_cube_size,
                'Clusters': len(df_clusters),
                'Mean - Vx': round(np.mean(df_clusters_est_err['vx_err[km/s]']),1), 
                'Mean - Vy': round(np.mean(df_clusters_est_err['vy_err[km/s]']),1), 
                'Mean - Vz': round(np.mean(df_clusters_est_err['vz_err[km/s]']),1), 
                'SD - Vx': round(np.std(df_clusters_est_err['vx_err[km/s]']),1), 
                'SD - Vy': round(np.std(df_clusters_est_err['vy_err[km/s]']),1),
                'SD - Vz': round(np.std(df_clusters_est_err['vz_err[km/s]']),1), 
                'r - Vx': round(np.corrcoef(df_clusters['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'])[1,0],2),
                'r - Vy': round(np.corrcoef(df_clusters['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'])[1,0],2),
                'r - Vz': round(np.corrcoef(df_clusters['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'])[1,0],2)}, 
                ignore_index=True)
        
        clus_cube_size = clus_cube_size_actual
        

**Writing & seeing the assesment table**

In [ ]:
cube_cell_size_assess.to_excel('cube_cell_size_assess.xlsx')
cube_cell_size_assess

**Reshaping the Pearson's r for Vz column into a more easy-to-read form**

In [ ]:
r_Vz_table = pd.DataFrame()

for name, group in cube_cell_size_assess.groupby("Cell Size"):
    if r_Vz_table.empty:
        r_Vz_table = group.set_index("Act Cube Size")[["r - Vz"]].rename(columns={"r - Vz":name})
    else:
        r_Vz_table = r_Vz_table.join(group.set_index("Act Cube Size")[["r - Vz"]].rename(columns={"r - Vz":name}))

r_Vz_table

**Plotting the mean, SD & r response due to variation of cell sizes for a given cluster cube size**

In [ ]:
x_axis = np.arange(0,6)

# for clus_cube_size in [240000]:
for clus_cube_size in [160000, 200000, 240000, 280000, 320000]:

    plt.figure(dpi = dpi, figsize = (5,9))
    plt.subplot(311)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Mean - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Mean - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['Mean - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cell size (h$^{-1}$kpc)')
    plt.ylabel('Mean of error (km/s)')
    plt.xticks(x_axis, [2000, 4000, 5000, 8000, 10000, 20000])
    minus_one = {-1}
    plt.title(r'Mean of velocity error | Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), fontsize = 11)

    plt.subplot(312)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['SD - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['SD - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['SD - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cell size (h$^{-1}$kpc)')
    plt.ylabel('SD of error (km/s)')
    plt.xticks(x_axis, [2000, 4000, 5000, 8000, 10000, 20000])
    minus_one = {-1}
    plt.title(r'SD of velocity error | Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), fontsize = 11)

    plt.subplot(313)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['r - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['r - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Act Cube Size'] == clus_cube_size]['r - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cell size (h$^{-1}$kpc)')
    plt.ylabel('Pearson\'s r')
    plt.xticks(x_axis, [2000, 4000, 5000, 8000, 10000, 20000])
    minus_one = {-1}
    plt.title(r"Pearson's r | Cube size: {} h$^{}$kpc".format(clus_cube_size, minus_one), fontsize = 11)
    plt.subplots_adjust(hspace = 0.6)
    plt.tight_layout()
    plt.savefig(f'Plots/cell_size_exam/clus_box_{clus_cube_size}.png')
    plt.close()

**Plotting the mean, SD & r response due to variation of cluster cube sizes for a given cell size**

In [ ]:

x_axis = np.arange(0,5)

# for cell_size in [5000]:
for cell_size in [2000, 4000, 5000, 8000, 10000, 20000]:

    plt.figure(dpi = dpi, figsize = (5,10))
    plt.subplot(311)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['Mean - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['Mean - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['Mean - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cube size (h$^{-1}$kpc)')
    plt.ylabel('Mean of error (km/s)')
    plt.xticks(x_axis, [160000, 200000, 240000, 280000, 320000])
    plt.ylim(-750,50)
    minus_one = {-1}
    plt.title(r'Mean of velocity error | Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), fontsize = 11)

    plt.subplot(312)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['SD - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['SD - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['SD - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cube size (h$^{-1}$kpc)')
    plt.ylabel('SD of error (km/s)')
    plt.ylim(100, 950)
    plt.xticks(x_axis, [160000, 200000, 240000, 280000, 320000])
    plt.title(r'SD of velocity error | Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), fontsize = 11)

    plt.subplot(313)
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['r - Vx'], label = 'Vx')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['r - Vy'], label = 'Vy')
    plt.plot(x_axis, cube_cell_size_assess[cube_cell_size_assess['Cell Size'] == cell_size]['r - Vz'], label = 'Vz')
    plt.legend()
    plt.xlabel(r'Cube size (h$^{-1}$kpc)')
    plt.ylabel('Pearson\'s r')
    plt.xticks(x_axis, [160000, 200000, 240000, 280000, 320000])
    plt.ylim(0.30, 0.90)
    plt.title(r"Pearson's r | Cell size: {} h$^{}$kpc".format(cell_size, minus_one), fontsize = 11)

    plt.subplots_adjust(hspace = 0.7)
    plt.tight_layout()
    plt.savefig(f'Plots/clus_bos_size_exam/cell_size_{cell_size}.png')
#     plt.savefig(f'Plots/clus_bos_size_exam/free_y_lim/cell_size_{cell_size}.png')
    plt.close()

**Obtaining velocity estimates for a given cell size & cluster box size**

In [ ]:
%%time
cell_size = 10000                      #h^-1 kpc, size of pixel or cell

clus_cube_size = 240000

no_of_cells = clus_cube_size//cell_size
if no_of_cells % 2 != 0:
    clus_cube_size = clus_cube_size + cell_size
    
df_clusters = edge_clus_remover(clus_cube_size)

sigma_in_pix = sigma_calc(cell_size)

delta_gal_mean = delta_gal_mean_func(cell_size)

vel_terms = vel_terms_calc(cell_size, clus_cube_size)

clus_param = list(zip(df_clusters['x[kpc/h]'], df_clusters['y[kpc/h]'], df_clusters['z[kpc/h]'], 
                      [delta_gal_mean]*len(df_clusters), [cell_size]*len(df_clusters), 
                      [vel_terms]*len(df_clusters), [sigma_in_pix]*len(df_clusters),
                      [clus_cube_size]*len(df_clusters)))

pool = multi.Pool(processes = cores)
v_est = pool.starmap(clus_velocity_calc, clus_param)

df_clusters_est_err = df_clusters.copy()

df_clusters_est_err['vx_est[km/s]'] = [i[0] for i in v_est]
df_clusters_est_err['vy_est[km/s]'] = [i[1] for i in v_est]
df_clusters_est_err['vz_est[km/s]'] = [i[2] for i in v_est]

df_clusters_est_err['vx_err[km/s]'] = df_clusters_est_err['vx[km/s]'] - df_clusters_est_err['vx_est[km/s]']
df_clusters_est_err['vy_err[km/s]'] = df_clusters_est_err['vy[km/s]'] - df_clusters_est_err['vy_est[km/s]']
df_clusters_est_err['vz_err[km/s]'] = df_clusters_est_err['vz[km/s]'] - df_clusters_est_err['vz_est[km/s]']

**Assessing the correctness of the estimated velocities**

In [ ]:
%%time

fig = plt.figure(dpi = dpi, figsize = (10,12))

plt.subplot(3,2,1)
plt.scatter(df_clusters_est_err['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'], s = 8)
plt.xlabel('V$_\mathrm{x, true}$ (km/s)')
plt.ylabel('V$_\mathrm{x, estimated}$ (km/s)')
plt.gca().set_xticks(range(-2000, 2001, 1000))
plt.gca().set_yticks(range(-2000, 2001, 1000))
plt.ylim(-2000,2000)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Vx - scatter plot')

plt.subplot(3,2,2)
error_x = df_clusters_est_err['vx[km/s]'] - df_clusters_est_err['vx_est[km/s]']
plt.hist(error_x, bins = 100)
plt.xlabel('Error in V$_{\mathrm{x}}$ (km/s)')
plt.ylabel('Number of clusters')
x_low, x_high = plt.xlim()
plt.xlim(x_low, abs(x_low))

minus_one = {-1}
plt.text(0.57, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes)
plt.text(0.57, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes)
plt.text(0.57, 0.76, f'Mean: {round(np.mean(error_x), 1)} km/s', transform=plt.gca().transAxes)
plt.text(0.57, 0.69, f'SD: {round(np.std(error_x), 1)} km/s', transform=plt.gca().transAxes)
r_vx = np.corrcoef(df_clusters_est_err['vx[km/s]'], df_clusters_est_err['vx_est[km/s]'])[1,0]
plt.text(0.57, 0.62, f'Pearson\'s r: {round(r_vx, 2)}', transform=plt.gca().transAxes)

plt.title('Vx error - histogram')




plt.subplot(3,2,3)
plt.scatter(df_clusters_est_err['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'], s = 8)
plt.xlabel('V$_\mathrm{y, true}$ (km/s)')
plt.ylabel('V$_\mathrm{y, estimated}$ (km/s)')
plt.gca().set_xticks(range(-2000, 2001, 1000))
plt.gca().set_yticks(range(-2000, 2001, 1000))
plt.ylim(-2000,2000)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Vy - scatter plot')

plt.subplot(3,2,4)
error_y = df_clusters_est_err['vy[km/s]'] - df_clusters_est_err['vy_est[km/s]']
plt.hist(error_y, bins = 100)
plt.xlabel('Error in V$_{\mathrm{y}}$ (km/s)')
plt.ylabel('Number of clusters')
x_low, x_high = plt.xlim()
plt.xlim(x_low, abs(x_low))

minus_one = {-1}
plt.text(0.57, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes)
plt.text(0.57, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes)
plt.text(0.57, 0.76, f'Mean: {round(np.mean(error_y), 1)} km/s', transform=plt.gca().transAxes)
plt.text(0.57, 0.69, f'SD: {round(np.std(error_y), 1)} km/s', transform=plt.gca().transAxes)
r_vy = np.corrcoef(df_clusters_est_err['vy[km/s]'], df_clusters_est_err['vy_est[km/s]'])[1,0]
plt.text(0.57, 0.62, f'Pearson\'s r: {round(r_vy, 2)}', transform=plt.gca().transAxes)

plt.title('Vy error - histogram')




plt.subplot(3,2,5)
plt.scatter(df_clusters_est_err['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'], s = 8)
plt.xlabel('V$_\mathrm{z, true}$ (km/s)')
plt.ylabel('V$_\mathrm{z, estimated}$ (km/s)');
plt.gca().set_xticks(range(-2000, 2001, 1000))
plt.gca().set_yticks(range(-2000, 2001, 1000))
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Vz - scatter plot')

plt.subplot(3,2,6)
error_z = df_clusters_est_err['vz[km/s]'] - df_clusters_est_err['vz_est[km/s]']
plt.hist(error_z, bins = 100)
plt.xlabel('Error in V$_{\mathrm{z}}$ (km/s)')
plt.ylabel('Number of clusters')
x_low, x_high = plt.xlim()
plt.xlim(x_low, abs(x_low))

minus_one = {-1}
plt.text(0.57, 0.90, r'Cell size: {} h$^{}$kpc'.format(cell_size, minus_one), transform=plt.gca().transAxes)
plt.text(0.57, 0.83, r'Cube size: {} h$^{}$kpc'.format(clus_cube_size, minus_one), transform=plt.gca().transAxes)
plt.text(0.57, 0.76, f'Mean: {round(np.mean(error_z), 1)} km/s', transform=plt.gca().transAxes)
plt.text(0.57, 0.69, f'SD: {round(np.std(error_z), 1)} km/s', transform=plt.gca().transAxes)
r_vz = np.corrcoef(df_clusters_est_err['vz[km/s]'], df_clusters_est_err['vz_est[km/s]'])[1,0]
plt.text(0.57, 0.62, f'Pearson\'s r: {round(r_vz, 2)}', transform=plt.gca().transAxes)

plt.title('Vz error - histogram')

# os.system(f'mkdir Plots/{clus_cube_size}')
plt.subplots_adjust(top = 0.9, hspace = 0.4, wspace = 0.3)
plt.tight_layout()
plt.close()

**Finding the minimum value of mean & Sd, and maximum value of r**

In [ ]:
print(cube_cell_size_assess['Mean - Vx'].abs().min())
print(cube_cell_size_assess['Mean - Vy'].abs().min())
print(cube_cell_size_assess['Mean - Vz'].abs().min())
print(cube_cell_size_assess['SD - Vx'].abs().min())
print(cube_cell_size_assess['SD - Vy'].abs().min())
print(cube_cell_size_assess['SD - Vz'].abs().min())
print(cube_cell_size_assess['r - Vx'].max())
print(cube_cell_size_assess['r - Vy'].max())
print(cube_cell_size_assess['r - Vz'].max())

**Conclusions**
1. All three components of velocities behave similarly so anyone of them can be taken from now onwards as the line of sight velocity. Pearson's r is highest (0.85) for V$_\mathrm{z}$ so maybe prefer V$_\mathrm{z}$ .
2. The code to estimate velocity seems to work fine since:  
(a) mean error in V$_\mathrm{x}$ goes to 0.3 km/s for cell size of 10,000 h$^\mathrm{-1}$kpc & cluster box size of 160,000 h$^\mathrm{-1}$kpc (SD is 185 km/s & r is 	0.78 for this case)   
(b) standard deviation of error in V$_\mathrm{z}$ goes to 146 km/s for cell size of 10,000 h$^\mathrm{-1}$kpc & cluster box size of 320,000 h$^\mathrm{-1}$kpc (Mean is -41.7 km/s & r is 0.85 for this case)  
(c) Pearson's r goes to 0.85 for V$_\mathrm{z}$ estimates coming from cell size of 10,000 h$^\mathrm{-1}$kpc & cluster box size of 280,000 h$^\mathrm{-1}$kpc (Mean is -43 km/s & SD is 152 km/s for this case)
3. Cell size of 10,000 h$^\mathrm{-1}$kpc should be chosen since it gives least mean, SD & maximum r.
4. Cluster box size does not make much difference but higher values do give a litter better mean, SD and r. A value of 280,000 h$^\mathrm{-1}$kpc may be choszen since it give r of 0.85.